# Implementing Basic Neural Networks with LENGTH

This is the first practical exercise of our course [Applied Edge AI](https://learn.ki-campus.org/courses/edgeai-hpi2022).
In this exercise, you are going to implement a few basic functions and library components of neural networks from scratch in Python.

Before we can actually start, install our pip package, which installs the surrounding library LENGTH (Lightning-fast Extensible Neural-network Guarding The HPI), by running the following cell (click the triangular "Play" button next to the cell or in the top bar).

In [ ]:
!pip install length_hpi
# if you get the warning "Failed to establish a new connection", go to the side bar on the right, then "Settings" and switch on "Internet"
# you can safely ignore errors such as "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. [...]"
# since we only need our length package, Pillow, and numpy in this exercise

Now we are going to prepare a few imports for the following code cells:

In [ ]:
import numpy as np

import length.functions as F

import length.tests.optimizer_tests.test_sgd as sgd_tests
import length.tests.layer_tests.test_fully_connected as fc_tests
import length.tests.function_tests.test_mean_squared_error as mse_tests
import length.tests.function_tests.test_relu as relu_tests

from length import constants
from length.data_sets import MNIST, FashionMNIST
from length.function import Function
from length.models import MLP
from length.optimizer import Optimizer

# Task 1: SGD

In one of the lectures in our course we learned about SGD.
In the following task we want to compute the parameter delta to actually implement SGD.
If you look up the formula on our course slides, the `param_deltas` are subtracted by our framework, thus we do *not* need to multiply our result with -1 in the code.
Also note, that the variable `gradients` is the *list* of computed derivatives, thus the derivative part of the formula is already computed here.

In [ ]:
class SGD(Optimizer):
    """
    An optimizer that does plain Stochastic Gradient Descent
    (https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method)
    :param lr: the learning rate to use for optimization
    """

    def __init__(self, lr):
        self.learning_rate = lr

    def run_update_rule(self, gradients, _):
        # :param gradients: a list of all computed gradient arrays
        # :return: a list of deltas for each provided gradient

        # TODO: implement SGD update rule and store the result in a variable called param_deltas (as a list)
        # HINT: it can be solved in a single line with a list comprehension ;)
        ### BEGIN SOLUTION
        #param_deltas = [self.learning_rate * gradient for gradient in gradients]
        ### END SOLUTION
        return param_deltas

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [ ]:
sgd_tests.SGD = SGD
sgd_tests.test_sgd()
print("Test passed.")

# Task 2: Fully Connected Layer

It seems we have not learned a lot about a "fully connected" layer (sometimes also called a "Dense" layer) in our lecture so far.
However we learned about Perceptrons and Multi Layer Perceptrons (MLPs), and fully connected layers are the building block of these early models for machine learning.

They simply store a weight between each possible input and output value and a bias for each output value.
For example, if we have 2 inputs $i_0,i_1$ and 3 outputs $o_0,o_1,o_2$, we store 6 weight values $w_{00}, w_{01}, w_{10}, w_{11}, w_{20},w_{21}$, one value for each pair of one input and one output, and three bias values $b_0,b_1,b_2$, one for each output.
Then during the forward pass the outputs of a fully connected layer are calculated as
$$o_x = \sum_{y=0}^1{(i_y \cdot w_{xy}) + b_x} \text{ for } 0 \leq x < 3.$$
This is simplified for a single element but in a neural network we work with mini-batches (processing a small number of samples at the same time).
In this case, we can use the [dot product](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) of two arrays to simplify this element-wise multiplication.

Here is a small very simple code snippet for the forward pass of our example case above with a batch size of 5.
The batch axis complicates matters slightly, but pay attention to the shape of each array and how the output size changes.
You can play around with different input and output sizes here if you want:

In [ ]:
batch_size = 5
num_inputs = 2
num_outputs = 3

i = np.arange(batch_size * num_inputs).reshape(batch_size, num_inputs)
w = np.arange(num_outputs * num_inputs).reshape(num_outputs, num_inputs)
b = np.zeros(num_outputs)
print(f"Inputs {i.shape}:\n", i)
print(f"Weights {w.shape}:\n", w)
print(f"Bias {b.shape}:\n", b)

output = np.dot(i, w.T) + b
print(f"Output {output.shape}:\n", output)

One interesting fact here can help you with implementing the backward pass:
applying the dot product on two arrays with shapes (5,2) and (2,3) "removes" the common axis with size 2 and results in an array of (5,3).
You can use this knowledge to figure out which arrays need to be transposed to get the correct shape during the *backward* pass.

Furthermore, you can recap how multiplication (between weights and inputs) affects the gradients in our lecture video [Computational Graph](https://learn.ki-campus.org/courses/edgeai-hpi2022/items/3btmrU8Ds8rVDk8SEUz1pU).
(Remember instead of using simple multiplication we can use the dot product for arrays.)
In the same video you can also recap what happens when we add *two* values in a computational graph (in this case the result of the dot product and our bias), so you can later on implement the *backward* pass for the *bias* correctly.

Also you may have noted, that we *transposed* the weight array in the example above - "swapping" the axes from (3,2) to (2,3) - with numpy before the dot product by calling `.T`.

This is because we recommend storing the weight array in a *transposed* way in your implementation (similar to our example above).

One final hint before you are ready to start implementing: we use the variable name `x` for the inputs (`grad_x` for the gradients with respect to the inputs) in our code below, instead of `i` in the example above (we only used `i` above so it maches the previous formula).

In [ ]:
from length.layer import Layer
from length.constants import DTYPE
from length.initializers.xavier import Xavier


class FullyConnected(Layer):
    """
    The FullyConnected Layer is one of the base building blocks of a neural network. It computes a weighted sum
    over the input, using a weight matrix. It furthermore applies a bias term to this weighted sum to allow linear
    shifts of the computed values.
    """
    name = "FullyConnected"

    def __init__(self, num_inputs, num_outputs, weight_init=Xavier()):
        super().__init__()

        # TODO: initialize our weights with correct shape, using the weight initializer 'weight_init'
        # here are two hints:
        # 1. store an array of zeros in `self._weights` with the correct shape (we recommend storing it transposed as in our simple example above) and use dtype=DTYPE
        # 2. call `weight_init` with our freshly created array `self._weights` to initialize the array properly
        ### BEGIN SOLUTION
        self._weights = np.zeros((num_outputs, num_inputs,), dtype=DTYPE)
        weight_init(self._weights)
        ### END SOLUTION
        
        # TODO: initialize `self.bias` with an array of zeros in the correct shape and use dtype=DTYPE
        ### BEGIN SOLUTION
        self.bias = np.zeros((num_outputs,), dtype=DTYPE)
        ### END SOLUTION

    @property
    def weights(self):
        # Transform weights between internal and external representation
        return self._weights.T

    @weights.setter
    def weights(self, value):
        # Transform weights between internal and external representation
        self._weights = value.T

    def internal_forward(self, inputs):
        x, = inputs
        
        # TODO: calculate the output of this layer and store it in a variable `result`
        #       (hint: you can look at our simple example above)
        ### BEGIN SOLUTION
        result = np.dot(x, self._weights.T) + self.bias
        ### END SOLUTION
        return result,

    def internal_backward(self, inputs, gradients):
        x, = inputs
        grad_in, = gradients
        
        # TODO: calculate gradients with respect to inputs for this layer
        # 1. calculate and store gradients for (the batch of) the inputs `x` in `grad_x`
        #    (hint: instead of simple multiplication we need to use the dot product for arrays)
        # 2. calculate and store gradients for the weights `w` in `grad_w`
        #    (hint: the shapes of `grad_w` and `self._weights` must be equal, so try to figure out which axes is "removed" by applying the dot product)
        # 3. calculate and store gradients for the bias `b` in `grad_b`
        #    (hint: gradients from multiple sources in the computational graph need to be added up)
        ### BEGIN SOLUTION
        grad_x = np.dot(grad_in, self._weights)
        grad_w = np.dot(grad_in.T, x)
        grad_b = np.sum(grad_in, axis=0)
        ### END SOLUTION

        assert grad_x.shape == x.shape
        assert grad_w.shape == self._weights.shape
        assert grad_b.shape == self.bias.shape

        return grad_x, grad_w, grad_b

    def internal_update(self, parameter_deltas):
        delta_w, delta_b = parameter_deltas
        
        # TODO: apply updates to weights (self._weights) and bias (self.bias) according to deltas from optimizer
        # if you remember our instructions on how to implement SGD, we said: "[...] the param_deltas are subtracted by our framework [...]"
        # so this is all we need to do here.
        ### BEGIN SOLUTION
        self._weights -= delta_w
        self.bias -= delta_b
        ### END SOLUTION

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [ ]:
fc_tests.FullyConnected = FullyConnected
fc_tests.test_initialization()
fc_tests.test_fully_connected_forward()
fc_tests.test_fully_connected_backward()
print("Test passed.")

# Task 3: Mean Squared Error

To train a model, we also need a loss function.
These loss functions mathematically define how our model should be optimized (and thus learn to solve a certain task).

A very simple loss function, which still can be effective in training models is the [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error).
Therefore in the following task, we are going to implement this function.
The corresponding [wikipedia article](https://en.wikipedia.org/wiki/Mean_squared_error) should explain everything you need to know, if you are not yet familiar with the function.

In [ ]:
class MeanSquaredError(Function):
    """
    This function calculates the Mean Squared Error between two given vectors, as described in
    https://en.wikipedia.org/wiki/Mean_squared_error
    """
    name = "MeanSquaredError"

    def __init__(self):
        super().__init__()
        # TODO: add more initialization if necessary
        ### BEGIN SOLUTION
        self.difference = None
        ### END SOLUTION

    @staticmethod
    def create_one_hot(data, shape):
        assert len(shape) == 2, "Providing integers as second input to MSE only works with two dimensional input vectors"
        # TODO: create a one-hot representation out of the given label vector (with dtype=DTYPE)
        # Example: assume `data` is [2, 3, 0], and the desired `shape` is (3, 4)
        #          in this case we have 4 possible classes and 3 samples, belonging to class 2, class 3, and class 0 
        #          therefore we need to set a 1 at position 2, 3, 0 for each sample respectively
        #          the resulting vector should look like this:
        #          result = [[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0]]
        # Hint: initialize an array of zeros with the given `shape`, set 1s where needed and in the end return your created array
        ### BEGIN SOLUTION
        data_container = np.zeros(shape, dtype=DTYPE)
        data_container[np.arange(len(data)), data] = 1        
        return data_container
        ### END SOLUTION

    def internal_forward(self, inputs):
        x1, x2 = inputs

        if np.issubdtype(x2.dtype, np.integer):
            x2 = self.create_one_hot(x2, x1.shape)

        # TODO: calculate the mean squared error of x1 and x2 and store the result in `mean_squared_error`
        # hint: you can store an intermediate result as a class member variable here that you need during the backward pass
        ### BEGIN SOLUTION
        #self.difference = x1 - x2
        squared_sum = np.sum(np.square(self.difference))
        mean_squared_error = (squared_sum / self.difference.size)
        ### END SOLUTION
        
        return mean_squared_error.astype(DTYPE),

    def internal_backward(self, inputs, gradients):
        x1, x2 = inputs
        gx, = gradients
        
        # TODO: calculate the gradients of this function with respect to its (two) inputs
        # (hint: the derivative depends on the inputs (here you could use an intermediate 
        #        result from the forward pass) and the size of the inputs)
        ### BEGIN SOLUTION
        derived_value = 2 / x1.size * self.difference
        gradient_1 = derived_value * gx
        gradient_2 = -gradient_1
        ### END SOLUTION

        if np.issubdtype(x2.dtype, np.integer):
            # in case we used MSE as loss function, we won't propagate any gradients to the loss
            return gradient_1, None

        return gradient_1, gradient_2


def mean_squared_error(input_1, input_2):
    """
    This function calculates the Mean Squared Error between input_1 and input_2. Both inputs should be vectors of the
    same shape. You can also supply a one-dimensional list of integers.
    If you do so this vector will be converted to a one_hot representation that fits to the shape of the second
    input
    :param input_1: the first vector of any shape
    :param input_2: the second vector. Needs to have the same shape as the first vector, or be a one-dimensional int vector
    :return: the mean squared error between input_1 and input_2
    """
    return MeanSquaredError()(input_1, input_2)

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [ ]:
mse_tests.MeanSquaredError = MeanSquaredError
mse_tests.mean_squared_error = mean_squared_error
mse_tests.test_mean_squared_error_forward_zero_loss()
mse_tests.test_mean_squared_error_forward_loss()
mse_tests.test_mean_squared_error_forward_int_input()
mse_tests.test_mean_squared_error_backward()
mse_tests.test_mean_squared_error_backward_with_label()
print("Test passed.")

# Task 4: ReLU (Rectified Linear Unit)

In our course we learned about how the ReLU function helped to solve the problem of vanishing gradients in the video [A Concise History of Neural Networks (3/4)](https://learn.ki-campus.org/courses/edgeai-hpi2022/items/22nlBMim7pwAX8A7gTI7qn).

In the next task, we are going to implement this function by filling in the missing forward and backward pass.

In [ ]:
class Relu(Function):
    """
    The Relu Layer is a non-linear activation
    """
    name = "ReLU"

    def __init__(self):
        super().__init__()
        # TODO: add more initialization of class member variables if necessary
        ### BEGIN SOLUTION
        self.output = None
        ### END SOLUTION

    def internal_forward(self, inputs):
        x, = inputs
        # TODO: calculate forward pass of ReLU function and store it in `activated_inputs`
        # (we can store any variables we need for the calculation of the backward pass in a class member variable here as well)
        ### BEGIN SOLUTION
        activated_inputs = np.maximum(x, np.zeros_like(x))
        self.output = activated_inputs
        ### END SOLUTION
        return activated_inputs,

    def internal_backward(self, inputs, gradients):
        x, = inputs
        grad_in, = gradients
        
        # TODO: calculate gradients of ReLU function with respect to the input and store it in grad_x
        # you can first calculate the derivative of ReLU itself (hint: it depends on the input of the forward pass)
        # and then use element-wise multiplication of the calculated derivative with the `grad_in` gradients
        ### BEGIN SOLUTION
        copy = self.output.copy()
        copy[copy > 0] = 1
        grad_x = np.multiply(grad_in, copy)
        ### END SOLUTION
        
        assert grad_x.shape == x.shape
        return grad_x,


def relu(x):
    """
    This function computes the element-wise ReLU activation function (https://en.wikipedia.org/wiki/Rectifier_(neural_networks))
    on a given input vector x.
    :param x: the input vector
    :return: a rectified version of the input vector
    """
    return Relu()(x)

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [ ]:
relu_tests.relu = relu
relu_tests.Relu = Relu
relu_tests.test_relu_forward()
relu_tests.test_relu_backward()
print("Test passed.")

# Check your score!

## Please follow these instructions before your run the below code cell.

- **Test case** code cells must not be changed or modified.
- You can run your tasks in any order but **ONLY** the below code cell should be executed at last. 
- Make sure you save (Ctrl + S) your notebook at regular intervals to keep your implementation(s) saved.
- Uncomment the below code to see results on this notebook.

**Note: If you see "Test passed." for all the tasks you get 4.0 points**

In [ ]:
#Uncomment & run the below code

#%run -m run
%run -i test_code_white.py "./ssnn_with_autograding_script.ipynb"